# Sistem de fuziune a deciziilor prin ponderare

In [1]:
from FusionSystem import *
import torch
import torch.optim as optim
import torch.nn as nn
from SegDataset import *
from utils.function_lib import *
from utils.DiceLoss import DiceLoss

In [2]:
training_path = 'G:/MachineLearning/liver-database/validation/'

training_dataset = SegDataset(training_path)
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)

device = torch.device('cuda')
fusion_model = FusionSystem(num_sources=4).to(device)
optimizer = optim.SGD(fusion_model.parameters() ,lr=0.5, momentum=0.9)
loss_fcn = DiceLoss().to(device)

In [3]:
for epoch in range(5):
    print(fusion_model.weights)
    train_loop(training_dataloader, fusion_model, loss_fcn, optimizer, scheduler=None,  device=device)
    loss = test_loop(training_dataloader, fusion_model, loss_fcn, device=device)

Parameter containing:
tensor([0.2500, 0.2500, 0.2500, 0.2500], device='cuda:0', requires_grad=True)
loss: 0.055459  [    0/ 4776]
loss: 0.058272  [ 1600/ 4776]
loss: 0.093113  [ 3200/ 4776]
Avg loss: 0.077708 

Parameter containing:
tensor([0.2500, 0.2500, 0.2500, 0.2500], device='cuda:0', requires_grad=True)
loss: 0.052392  [    0/ 4776]


KeyboardInterrupt: 